# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


In [2]:
insur = pd.read_csv('./insurance.csv')
insur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, нужно поменять тип данных в столбцах возвраст и зарплата на int

In [3]:
insur['Возраст'] = insur['Возраст'].astype(int)
insur['Зарплата'] = insur['Зарплата'].astype(int)
insur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


In [4]:
def info(data):
    for column in data.columns:
        print('Уникальные значения столбца', column)
        print(data[column].unique())

In [5]:
info(insur)

Уникальные значения столбца Пол
[1 0]
Уникальные значения столбца Возраст
[41 46 29 21 28 43 39 25 36 32 38 23 40 34 26 42 27 33 47 30 19 31 22 20
 24 18 37 48 45 44 52 49 35 56 65 55 57 54 50 53 51 58 59 60 61 62]
Уникальные значения столбца Зарплата
[49600 38000 21000 41700 26100 41000 39700 38600 49700 51700 36600 29300
 39500 55000 43700 23300 48900 33200 36900 43500 36100 26600 48700 40400
 38400 34600 34800 36800 42200 46300 30300 51000 28100 64800 30400 45300
 38300 49500 19400 40200 31700 69200 33100 31600 34500 38700 39600 42400
 34900 30500 24200 49900 14300 47000 44800 43800 42700 35400 57200 29600
 37400 48100 33700 61800 39400 15600 52600 37600 52500 32700 51600 60900
 41800 47400 26500 45900 35700 34300 26700 25700 33300 31100 31500 42100
 37300 42500 27300 46800 33500 44300 41600 53900 40100 44600 45000 32000
 38200 33000 38500 51800 33800 46400 43200 31800 50200 35100 30700 45800
 49300 42800 33600 50300 34000 36400 44900 43600 54600 52800 29700 39000
 44100 31900 37700

Пропусков нет, тип данных в столбцах "Возраст" и "Зарплата" поменяли на int, чтобы данные были целочисленные.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Вопрос: признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:** 

Заменим матрицу X на XP и вычислим: 
    
$$
w = ((XP)^T (XP))^{-1} (XP)^T y
$$

Подставим в уравнение $a$ =XPw:
    
$$
a_{1} = XP((XP)^T (XP))^{-1} (XP)^T y 
$$

Применим свойство обратной матрицы, но для начало раскроем скобки $((XP)^T (XP))^{-1}$:

$$
(AB)^{-1} = B^{-1}A^{-1}
$$

$$
a_{1} = XP((XP)^T (XP))^{-1}(XP)^T y = XP(P^T (X^T X) P)^{-1}(XP)^T y = XP P^{-1} (X^T X)^{-1} (P^T)^{-1}(XP)^T y =|PP^{-1}=E|= XE (X^T X)^{-1}(P^T)^{-1}(XP)^T y
$$

Теперь воспользуемся свойством транспанированной матрицы и снова свойством обратной:
    
$$
(AB)^{T} = B^TA^T
$$

$$
a_{1} = X (X^T X)^{-1}(P^T)^{-1}(XP)^T y = X (X^T X)^{-1}(P^T)^{-1} P^T X^T y=|(P^T)^{-1}P^T=E|= X (X^T X)^{-1} X^T y
$$

Получаем:

$$
a_{1} = X (X^T X)^{-1} X^T y = Xw = a
$$

Таким образом, при умножении матрицы признаков на обратимую матрицу, качество линейной регрессии не изменится. Что и требовалось доказать!

## Алгоритм преобразования

**Алгоритм**

Для защиты информации будем умножать матрицу признаков на обратимую матрицу, которая будет генерироваться случайным образом.

1. Составим рандомную матрицу Y;
2. Проверим эту матрицу на обратимость;
3. Получим матрицу преобразованных признаков Z=X*Y.

**Обоснование**

Матрица Y должна иметь размерность $nxn$, где n - количество признаков для регрессии. Таким образом матрица Z будет иметь туже размерность, что и матрица X. Обратная матрица Y существует только для квадратных матриц (определитель которых не равен нулю).

## Проверка алгоритма

Вначале найдем качество линейной регрессии до преобразование признаков

In [6]:
#выберем признаки
features = insur.drop('Страховые выплаты',axis=1)
target = insur['Страховые выплаты']

features.shape, target.shape

((5000, 4), (5000,))

In [7]:
#напишем функцию для подсчета качества модели линейной регрессии
def score_mod(features, target):
    model = LinearRegression()
    model.fit(features, target)
    predictions = model.predict(features)

    print(r2_score(target, predictions))

In [8]:
#качество обычной матрицы до преобразования:
score_mod(features, target)

0.42494550308169177


Теперь найдем качество после преобразование признаков

In [9]:
#создадим рандомную матрицу 4х4
rnd_matrix = np.random.rand(4,4)
rnd_matrix

array([[0.48940848, 0.87995235, 0.94838183, 0.2642182 ],
       [0.54144805, 0.84685462, 0.71118365, 0.21583467],
       [0.32788276, 0.04337465, 0.52614921, 0.59230732],
       [0.03943066, 0.87596801, 0.51377812, 0.45952607]])

In [10]:
#проверим на обратимость
inv = np.linalg.inv(rnd_matrix)
inv

array([[-2.76130136,  4.02804901,  0.65842199, -1.15291554],
       [-1.23307307,  1.49692365, -0.74808287,  0.97014577],
       [ 4.21580249, -3.74184716, -0.03953434, -0.61553613],
       [-2.12604826,  0.98447809,  1.4137316 ,  1.11395761]])

In [11]:
#умножили признаки на рандомную матрицу
features_new = features @ rnd_matrix
features_new.head()

,0,1,2,3
0,16285.712909,2187.859364,26127.621466,29388.015907
1,12484.490771,1688.067800,20026.898176,22518.065981
2,6901.239871,935.426334,11069.757719,12444.712870
3,13684.160198,1828.258590,21956.384436,24704.666714
4,8573.389887,1156.670123,13753.355884,15465.528572


In [12]:
#качество модели на преобразованных признаках
score_mod(features_new, target)

0.42494550308172163


In [13]:
#расшифровка
descript = features_new @ inv
descript.head()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-7.275958e-12,46.0,38000.0,1.000000e+00
2,3.637979e-12,29.0,21000.0,0.000000e+00
3,-1.455192e-11,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,3.637979e-12


Вывод:

- изучили данные, провели предобработку данных;
- доказали, что при умножении матрицы на обратимую, качество не поменяется;
- написали алгоритм преобразования;
- проверили алгоритм до преобразования данных и после, качество модели не изменилось 0.4249455